<a href="https://colab.research.google.com/github/syedahafsa12/Langgraph_Projects/blob/main/HumanInTheLoop_Langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_google_genai
!pip install --quiet langgraph langchain-google-genai langchain_core python_dotenv
!pip install langgraph
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
!pip install langgraph langchain langchain_google_genai





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00


In [ ]:
# 📌 REQUIRED LIBRARIES
# -------------------------------
import os
import requests
from typing import TypedDict
from google.colab import userdata
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

# -------------------------------
# 📌 SETUP AND API CONFIGURATION
# -------------------------------

# ⚠️ API KEYS (HARD-CODED FOR TEACHER EVALUATION PURPOSES ONLY)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
SEARCH_ENGINE_ID = userdata.get('SEARCH_ENGINE_ID')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Validate API keys
if not GOOGLE_API_KEY or not SEARCH_ENGINE_ID or not GEMINI_API_KEY:
    raise ValueError("❌ Missing API keys. Please set GOOGLE_API_KEY, SEARCH_ENGINE_ID, and GEMINI_API_KEY.")

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GEMINI_API_KEY)

# -------------------------------
# 📌 STATE DEFINITION
# -------------------------------
class ChefMateState(TypedDict):
    user_info: dict
    messages: list
    introduced: bool

# -------------------------------
# 📌 HELPER FUNCTIONS
# -------------------------------

FAILURE_CLUES = [
    'i need more information',
    'i do not have enough details',
    'i cannot',
    'i’m not sure',
    'consult a human',
    'please be more specific',
    'unable to do that',
    'i can’t calculate',
    'i can’t',
    'i don’t know',
    'help'
]

def check_for_gemini_failure(response: str) -> bool:
    """Check if Gemini's response contains any failure clues."""
    for clue in FAILURE_CLUES:
        if clue in response.lower():
            print("\n🛑 **I'm connecting you to a human assistant for this question...**\n")
            return True
    return False

def search_pakistani_dishes_on_google(ingredients: str, meal_type: str):
    """Search for Pakistani dishes using Google Custom Search API."""
    print(f"🔍 Searching Google for: Pakistani {meal_type} dishes with {ingredients}...")
    query = f"Pakistani {meal_type} dishes with {ingredients}"
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&cx={SEARCH_ENGINE_ID}&key={GOOGLE_API_KEY}&searchType=image&num=5"
    try:
        response = requests.get(url)
        response.raise_for_status()
        search_results = response.json().get("items", [])
        print(f"✅ Found {len(search_results)} dishes from Google.")
        return search_results
    except Exception as e:
        print(f"❌ Error searching dishes on Google: {e}")
        return []

def display_meal_ideas(meals):
    """Display the top 5 meal ideas with title, image, and recipe URL."""
    if not meals:
        print("❌ No meals available.")
        return

    print("\n🍽️ Here are some meal ideas for you:\n")
    for idx, meal in enumerate(meals[:5]):
        title = meal.get("title", "Meal Idea")
        image = meal.get("image", {}).get("thumbnailLink", "No image available")
        recipe_link = meal.get("image", {}).get("contextLink", "No recipe link available")
        print(f"{idx + 1}. 🍽️ {title}")
        print(f"   📷 Image: {image}")
        print(f"   🔗 Recipe Link: {recipe_link}\n")

# -------------------------------
# 📌 NODES (STEPS OF THE CHATBOT)
# -------------------------------

def introduce_chefmate(state: ChefMateState):
    """Introduce the chatbot to the user."""
    if not state.get("introduced"):
        print("\n🤖 Hi! I’m ChefMate, your smart recipe assistant. 🍳")
        state["introduced"] = True
    return state

def ask_for_ingredients(state: ChefMateState):
    """Ask the user for ingredients available."""
    ingredients = input("👤 What's in your fridge? (List your ingredients separated by commas): ")
    if not ingredients:
        print("❌ No ingredients provided, using default ingredients 'chicken, rice'.")
        ingredients = "chicken, rice"
    state["user_info"] = {"ingredients": ingredients}
    return state

def ask_for_meal_type(state: ChefMateState):
    """Ask the user for the type of meal they are looking for."""
    meal_type = input("👤 What type of meal are you looking for? (Breakfast, Lunch, Dinner, Drinks): ")
    if not meal_type:
        print("❌ No meal type provided, defaulting to 'dinner'.")
        meal_type = "dinner"
    state["user_info"]["meal_type"] = meal_type
    return state

def fetch_meal_ideas(state: ChefMateState):
    """Fetch meal ideas based on user input."""
    ingredients = state["user_info"].get("ingredients", "")
    meal_type = state["user_info"].get("meal_type", "dinner")
    print(f"\n🍽️ Searching for meal ideas for: {ingredients} ({meal_type})... Please wait...\n")
    meals = search_pakistani_dishes_on_google(ingredients, meal_type)
    if meals:
        display_meal_ideas(meals)
    else:
        print(f"❌ No meal ideas found on Google for ingredients: {ingredients}.")
    return state

def chat_with_gemini(state: ChefMateState):
    """Chat with Gemini, handle dynamic interrupts and human assistant interruptions."""
    print("\n💬 **You can now chat with me about the meal ideas! Type 'exit' to quit.**\n")
    while True:
        user_input = input("💬 **You:** ")
        if 'help' in user_input.lower():
            print("\n🛑 **I'm connecting you to a human assistant for this request...**\n")
            break
        if user_input.lower() in ['exit', 'quit']:
            print("👋 **Bye! See you next time!**")
            break
        if len(state["messages"]) > 10:
            state["messages"].pop(0)
        state["messages"].append({"role": "user", "content": user_input})
        response = llm.invoke([{"role": "user", "content": f"Context: {user_input}"}])
        print(f"🤖 {response.content}")
        if check_for_gemini_failure(response.content):
            break
        state["messages"].append({"role": "bot", "content": response.content})
    return state

# -------------------------------
# 📌 LANGGRAPH FLOW DEFINITION
# -------------------------------

builder = StateGraph(ChefMateState)
builder.add_node("introduce_chefmate", introduce_chefmate)
builder.add_node("ask_for_ingredients", ask_for_ingredients)
builder.add_node("ask_for_meal_type", ask_for_meal_type)
builder.add_node("fetch_meal_ideas", fetch_meal_ideas)
builder.add_node("chat_with_gemini", chat_with_gemini)

builder.add_edge(START, "introduce_chefmate")
builder.add_edge("introduce_chefmate", "ask_for_ingredients")
builder.add_edge("ask_for_ingredients", "ask_for_meal_type")
builder.add_edge("ask_for_meal_type", "fetch_meal_ideas")
builder.add_edge("fetch_meal_ideas", "chat_with_gemini")

graph = builder.compile()

# -------------------------------
# 📌 MAIN CHAT LOOP
# -------------------------------
if __name__ == "__main__":
    print("🍳 Welcome to ChefMate CLI! Type 'exit' to quit.\n")
    state = {"user_info": {}, "messages": [], "introduced": False}
    state = graph.invoke(state)

🍳 Welcome to ChefMate CLI! Type 'exit' to quit.


🤖 Hi! I’m ChefMate, your smart recipe assistant. 🍳
👤 What's in your fridge? (List your ingredients separated by commas): chicken
👤 What type of meal are you looking for? (Breakfast, Lunch, Dinner, Drinks): lunch

🍽️ Searching for meal ideas for: chicken (lunch)... Please wait...

🔍 Searching Google for: Pakistani lunch dishes with chicken...
✅ Found 5 dishes from Google.

🍽️ Here are some meal ideas for you:

1. 🍽️ 9 Special Pakistani Chicken Recipes by (YES I CAN COOK) - YouTube
   📷 Image: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQv1mmixQZKnhjLIetS_CVPQNTdrXy5VcwoQVS2mOjJISdDvnu_X1pId3U&s
   🔗 Recipe Link: https://www.youtube.com/watch?v=l5EUP7Z3gqM

2. 🍽️ Instant Pot Pakistani Chicken Curry with Potatoes - Tea for Turmeric
   📷 Image: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSpN5hraoaPNCW0sTDQTuVtj6cy2rttmxObA7K4mAR4oodGq-5oD4oOXOA&s
   🔗 Recipe Link: https://www.teaforturmeric.com/instant-pot-chicken-curry/